In [1]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from dotenv import load_dotenv
import chromadb
import json
import os
import pandas as pd


In [2]:
def create_skills_matrix_with_distances(skills_dict):
    """
    Create a DataFrame showing the best (smallest) distance scores between entities and skills 
    based on ChromaDB similarity search.
    
    Args:
        skills_dict (dict): Dictionary of skills categorized by priority level
                           Format: {
                               'Critical': ['skill1', 'skill2'],
                               'Required': ['skill3', 'skill4'],
                               'Preferred': ['skill5', 'skill6'],
                               'Optional': ['skill7', 'skill8']
                           }
        
    Returns:
        pandas.DataFrame: Matrix of entities and their best skill distances with priority level prefixes
    """
    # Load environment variables
    load_dotenv()

    # Initialize ChromaDB client with persistence
    client = chromadb.PersistentClient(path="../entity_skills_db")

    # Initialize the OpenAI embedding function
    embedding_function = OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-3-large"
    )

    # Get existing collection
    collection = client.get_collection(
        name="entity_skills",
        embedding_function=embedding_function
    )

    # Get the total count of items in the collection
    collection_size = collection.count()

    # Dictionary to store all results
    all_entity_skills = {}
    
    # Create a mapping of all skills to their priority levels
    skill_priority_map = {
        skill: priority
        for priority, skills in skills_dict.items()
        for skill in skills
    }
    
    # Get all skills across all priority levels
    all_skills = [skill for skills in skills_dict.values() for skill in skills]
    
    # Query each skill
    for skill in all_skills:
        results = collection.query(
            query_texts=[skill],
            n_results=collection_size,
            include=["documents", "metadatas", "distances"]
        )
        
        # Process results for this skill
        for entity_metadata, distance in zip(
            results['metadatas'][0],
            results['distances'][0]
        ):
            entity_id = entity_metadata.get('entity_name')
            
            # Initialize dictionary for new entity if needed
            if entity_id not in all_entity_skills:
                all_entity_skills[entity_id] = {}
            
            # Get the column name
            priority = skill_priority_map[skill]
            column_name = f"{priority}_{skill}"
            
            # Update the distance if it's either not set yet or if this one is smaller
            current_distance = all_entity_skills[entity_id].get(column_name, None)
            if current_distance is None or distance < current_distance:
                all_entity_skills[entity_id][column_name] = round(distance, 2)
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(all_entity_skills, orient='index')
    
    # Reset index and rename it to entity_id
    df.index.name = 'entity_id'
    df.reset_index(inplace=True)
    
    return df

In [3]:
def calculate_scores(df):
    """
    Calculate scores for priority columns while preserving other columns.
    Adds total score and rank columns, sorts by total score descending.
    
    Rules:
    - 'Critical' prefix & True value = 20
    - 'Required' prefix & True value = 10
    - 'Preferred' prefix & True value = 7
    - 'Optional' prefix & True value = 3
    - All other columns preserved as-is
    - 'total_priority_score' column added with sum of priority scores
    - 'rank' column added based on total_priority_score
    - Result sorted by total_priority_score descending
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame with boolean columns
    
    Returns:
    pandas.DataFrame: New DataFrame with scores, total, and rank
    """
    # Define priority prefixes
    priority_prefixes = ['Critical', 'Required', 'Preferred', 'Optional']
    
    # Create a copy of the input DataFrame
    result_df = df.copy()
    
    # Track which columns are priority columns for summing later
    priority_columns = []
    
    # Process each column in the source DataFrame
    for col in df.columns:
        # Check if column starts with any of our priority prefixes
        if any(col.startswith(prefix) for prefix in priority_prefixes):
            new_values = pd.Series(0, index=df.index)
            
            if col.startswith('Critical'):
                new_values[df[col]] = 20
            elif col.startswith('Required'):
                new_values[df[col]] = 10
            elif col.startswith('Preferred'):
                new_values[df[col]] = 7
            elif col.startswith('Optional'):
                new_values[df[col]] = 3
                
            result_df[col] = new_values
            priority_columns.append(col)
    
    # Add total column that sums only priority-based scores
    result_df['total_priority_score'] = result_df[priority_columns].sum(axis=1)
    
    # Sort by total_priority_score descending
    result_df = result_df.sort_values('total_priority_score', ascending=False)
    
    # Add rank column (1-based ranking)
    result_df['rank'] = range(1, len(result_df) + 1)
    
    # Reorder columns to put rank first, then entity_id, then rest
    cols = ['rank', 'entity_id'] + [col for col in result_df.columns if col not in ['rank', 'entity_id']]
    result_df = result_df[cols]
    
    return result_df

In [4]:
def create_skills_matrix(skills_dict, distance_threshold=.8):
    """
    Create a DataFrame showing which entities possess which skills based on ChromaDB similarity search.
    Skills are categorized by priority level and reflected in column names.
    
    Args:
        skills_dict (dict): Dictionary of skills categorized by priority level
                           Format: {
                               'Critical': ['skill1', 'skill2'],
                               'Required': ['skill3', 'skill4'],
                               'Preferred': ['skill5', 'skill6'],
                               'Optional': ['skill7', 'skill8']
                           }
        distance_threshold (float): Maximum distance to consider a skill match (default: .8)
        
    Returns:
        pandas.DataFrame: Matrix of entities and their skills with priority level prefixes
    """
    # Load environment variables
    load_dotenv()

    # Initialize ChromaDB client with persistence
    client = chromadb.PersistentClient(path="../entity_skills_db")

    # Initialize the OpenAI embedding function
    embedding_function = OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-3-large"
    )

    try:
        # Get existing collection
        collection = client.get_collection(
            name="entity_skills",
            embedding_function=embedding_function
        )

        # Get the total count of items in the collection
        collection_size = collection.count()
        
        # Dictionary to store all results
        all_entity_skills = {}
        
        # Create a mapping of all skills to their priority levels
        skill_priority_map = {
            skill: priority
            for priority, skills in skills_dict.items()
            for skill in skills
        }
        
        # Get all skills across all priority levels
        all_skills = [skill for skills in skills_dict.values() for skill in skills]
        
        # Query each skill
        for skill in all_skills:
            results = collection.query(
                query_texts=[skill],
                n_results=collection_size,
                include=["documents", "metadatas", "distances"]
            )
            
            # Process results for this skill
            for entity_metadata, distance in zip(
                results['metadatas'][0],
                results['distances'][0]
            ):
                entity_id = entity_metadata.get('entity_name')
                
                # Initialize entity in dictionary if not present
                if entity_id not in all_entity_skills:
                    # Initialize with prefixed column names
                    all_entity_skills[entity_id] = {
                        f"{priority}_{skill}": False
                        for priority, skills in skills_dict.items()
                        for skill in skills
                    }
                
                # Mark skill as True if distance is below threshold
                if distance < distance_threshold:
                    priority = skill_priority_map[skill]
                    all_entity_skills[entity_id][f"{priority}_{skill}"] = True
        
        # Convert to DataFrame
        df = pd.DataFrame.from_dict(all_entity_skills, orient='index')
        
        # Reset index and rename it to entity_id
        df.index.name = 'entity_id'
        df.reset_index(inplace=True)
        
        return df

    except Exception as e:
        print(f"Error accessing collection: {str(e)}")
        raise

# Example usage
# if __name__ == "__main__":
#     # Example dictionary of skills to query
#     skills_to_query = {
#         'Critical': ['Python', 'Data Analysis'],
#         'Required': ['Amazon Web Services', 'Machine Learning', 'PyTorch'],
#         'Preferred': ['Docker', 'SQL', 'SQL Server', 'PostgreSQL'],
#         'Optional': ['Kubernetes', 'React', 'GCP'],
#     }
    
#     # Create the skills matrix
#     skills_df = create_skills_matrix(skills_to_query)

    
    # # Display some summary statistics
    # print("\nSkill Distribution:")
    # for priority, skills in skills_to_query.items():
    #     print(f"\n{priority} Skills:")
    #     for skill in skills:
    #         column_name = f"{priority}_{skill}"
    #         count = skills_df[column_name].sum()
    #         total = len(skills_df)
    #         percentage = (count / total) * 100
    #         print(f"{skill}: {count} entities ({percentage:.1f}%)")



In [5]:
def create_skills_skill_matrix(skills_dict, distance_threshold=.8):
    """
    Create a DataFrame showing which entities possess which skills based on ChromaDB similarity search.
    Returns the actual matching skill text from the database instead of boolean values.
    
    Args:
        skills_dict (dict): Dictionary of skills categorized by priority level
                           Format: {
                               'Critical': ['skill1', 'skill2'],
                               'Required': ['skill3', 'skill4'],
                               'Preferred': ['skill5', 'skill6'],
                               'Optional': ['skill7', 'skill8']
                           }
        distance_threshold (float): Maximum distance to consider a skill match (default: .8)
        
    Returns:
        pandas.DataFrame: Matrix of entities and their skills with priority level prefixes,
                        containing the actual matching skill text instead of boolean values
    """
    # Load environment variables
    load_dotenv()

    # Initialize ChromaDB client with persistence
    client = chromadb.PersistentClient(path="../entity_skills_db")

    # Initialize the OpenAI embedding function
    embedding_function = OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-3-large"
    )

    try:
        # Get existing collection
        collection = client.get_collection(
            name="entity_skills",
            embedding_function=embedding_function
        )

        # Get the total count of items in the collection
        collection_size = collection.count()
        
        # Dictionary to store all results
        all_entity_skills = {}
        
        # Create a mapping of all skills to their priority levels
        skill_priority_map = {
            skill: priority
            for priority, skills in skills_dict.items()
            for skill in skills
        }
        
        # Get all skills across all priority levels
        all_skills = [skill for skills in skills_dict.values() for skill in skills]
        
        # Query each skill
        for skill in all_skills:
            results = collection.query(
                query_texts=[skill],
                n_results=collection_size,
                include=["documents", "metadatas", "distances"]
            )
            
            # Process results for this skill
            for doc, entity_metadata, distance in zip(
                results['documents'][0],
                results['metadatas'][0],
                results['distances'][0]
            ):
                entity_id = entity_metadata.get('entity_name')
                
                # Initialize entity in dictionary if not present
                if entity_id not in all_entity_skills:
                    # Initialize with empty lists for each skill
                    all_entity_skills[entity_id] = {
                        f"{priority}_{skill}": []
                        for priority, skills in skills_dict.items()
                        for skill in skills
                    }
                
                # Add skill text to list if distance is below threshold
                if distance < distance_threshold:
                    priority = skill_priority_map[skill]
                    all_entity_skills[entity_id][f"{priority}_{skill}"].append(doc)
        
        # Convert lists to comma-separated strings
        for entity_id in all_entity_skills:
            for column in all_entity_skills[entity_id]:
                skills_list = all_entity_skills[entity_id][column]
                all_entity_skills[entity_id][column] = '; '.join(skills_list) if skills_list else ''
        
        # Convert to DataFrame
        df = pd.DataFrame.from_dict(all_entity_skills, orient='index')
        
        # Reset index and rename it to entity_id
        df.index.name = 'entity_id'
        df.reset_index(inplace=True)
        
        return df

    except Exception as e:
        print(f"Error accessing collection: {str(e)}")
        raise

# Example usage
# if __name__ == "__main__":
#     # Example dictionary of skills to query
#     skills_to_query = {
#         'Critical': ['Python', 'Data Analysis'],
#         'Required': ['Amazon Web Services', 'Machine Learning', 'PyTorch'],
#         'Preferred': ['Docker', 'SQL', 'SQL Server', 'PostgreSQL'],
#         'Optional': ['Kubernetes', 'React', 'GCP'],
#     }
    
#     # Create the skills matrix
#     skills_df = create_skills_matrix(skills_to_query)
    
#     # Display some summary statistics
#     print("\nSkill Distribution:")
#     for priority, skills in skills_to_query.items():
#         print(f"\n{priority} Skills:")
#         for skill in skills:
#             column_name = f"{priority}_{skill}"
#             # Count non-empty entries
#             count = skills_df[column_name].str.len().gt(0).sum()
#             total = len(skills_df)
#             percentage = (count / total) * 100
#             print(f"{skill}: {count} entities ({percentage:.1f}%)")
#             
#             # Display some example matches
#             matches = skills_df[skills_df[column_name].str.len() > 0][column_name].head()
#             if not matches.empty:
#                 print("Sample matches:")
#                 for match in matches:
#                     print(f"  - {match}")

In [6]:
skills_to_query = {
    'Critical': ['Python', 'PyTorch', 'GCP', 'Cloud Services'],
    'Required': ['Amazon Web Services', 'Machine Learning'],
    'Preferred': ['Docker', 'SQL', 'SQL Server', 'PostgreSQL', 'Object Oriented Programming'],
    'Optional': ['Kubernetes', 'React'],
}

# Create the skills matrix
skills_df = create_skills_matrix(skills_to_query)
skills_df

,entity_id,Critical_Python,Critical_PyTorch,Critical_GCP,Critical_Cloud Services,Required_Amazon Web Services,Required_Machine Learning,Preferred_Docker,Preferred_SQL,Preferred_SQL Server,Preferred_PostgreSQL,Preferred_Object Oriented Programming,Optional_Kubernetes,Optional_React
0,Carol Wong,True,True,True,True,False,True,False,False,False,False,True,False,False
1,Alice Chen,True,False,False,True,True,False,False,False,True,False,False,True,False
2,Bob Martinez,False,False,False,True,True,False,False,False,False,True,True,False,True


In [7]:
skill_skill_df = create_skills_skill_matrix(skills_to_query)
skill_skill_df

,entity_id,Critical_Python,Critical_PyTorch,Critical_GCP,Critical_Cloud Services,Required_Amazon Web Services,Required_Machine Learning,Preferred_Docker,Preferred_SQL,Preferred_SQL Server,Preferred_PostgreSQL,Preferred_Object Oriented Programming,Optional_Kubernetes,Optional_React
0,Carol Wong,Python,PyTorch,Google Cloud,Google Cloud,,Machine Learning,,,,,Python Object Oriented Programming,,
1,Alice Chen,Python Programming,,,cloud platform,AWS,,,,SQLServer,,,K8s,
2,Bob Martinez,,,,cloud service,AWS,,,,,PostrgreSQL,OOP,,React


In [8]:
scored_df = calculate_scores(skills_df)
scored_df

,rank,entity_id,Critical_Python,Critical_PyTorch,Critical_GCP,Critical_Cloud Services,Required_Amazon Web Services,Required_Machine Learning,Preferred_Docker,Preferred_SQL,Preferred_SQL Server,Preferred_PostgreSQL,Preferred_Object Oriented Programming,Optional_Kubernetes,Optional_React,total_priority_score
0,1,Carol Wong,20,20,20,20,0,10,0,0,0,0,7,0,0,97
1,2,Alice Chen,20,0,0,20,10,0,0,0,7,0,0,3,0,60
2,3,Bob Martinez,0,0,0,20,10,0,0,0,0,7,7,0,3,47


In [9]:
skills_distances_df = create_skills_matrix_with_distances(skills_to_query)
skills_distances_df

,entity_id,Critical_Python,Critical_PyTorch,Critical_GCP,Critical_Cloud Services,Required_Amazon Web Services,Required_Machine Learning,Preferred_Docker,Preferred_SQL,Preferred_SQL Server,Preferred_PostgreSQL,Preferred_Object Oriented Programming,Optional_Kubernetes,Optional_React
0,Carol Wong,0.00,0.00,0.69,0.69,0.94,0.00,1.29,1.25,1.20,1.36,0.62,1.01,1.34
1,Alice Chen,0.72,0.85,0.97,0.58,0.55,1.06,1.02,0.82,0.17,1.12,1.09,0.42,1.35
2,Bob Martinez,0.86,1.31,1.10,0.30,0.55,1.40,0.82,0.98,1.05,0.08,0.59,1.15,0.00


In [10]:
# skills_to_query = {
#     'Critical': ['Python Programming', 'Data Analysis'],
#     'Required': ['Amazon Web Services', 'Machine Learning', 'PyTorch'],
#     'Preferred': ['Docker', 'SQL', 'SQL Server', 'PostgreSQL'],
#     'Optional': ['Kubernetes', 'React', 'GCP', 'Google Cloud', 'Google Cloud Platform'],
# }

# Create the skills matrix with distances


# # Display some summary statistics
# print("\nSkill Distance Statistics:")
# for priority, skills in skills_to_query.items():
#     print(f"\n{priority} Skills:")
#     for skill in skills:
#         column_name = f"{priority}_{skill}"
#         mean_distance = skills_distances_df[column_name].mean()
#         close_matches = (skills_distances_df[column_name] < 0.8).sum()
#         total = len(skills_distances_df)
#         percentage = (close_matches / total) * 100
#         print(f"{skill}:")
#         print(f"  Mean distance: {mean_distance:.2f}")
#         print(f"  Close matches (<0.8): {close_matches} entities ({percentage:.1f}%)")
        
# # Optional: Display the top 5 closest matches for each skill
# print("\nTop 5 Closest Matches by Skill:")
# for priority, skills in skills_to_query.items():
#     print(f"\n{priority} Skills:")
#     for skill in skills:
#         column_name = f"{priority}_{skill}"
#         print(f"\n{skill}:")
#         top_5 = skills_distances_df.nsmallest(5, column_name)[['entity_id', column_name]]
#         print(top_5.to_string(index=False))